In [0]:
from pyspark.sql.functions import * 
from pyspark.sql.window import *

In [0]:
pip install openpyxl

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.0/250.0 kB 741.4 kB/s eta 0:00:00
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [0]:
import pandas as pd

# Read the Excel file using pandas
df = pd.read_excel('/dbfs/FileStore/pei/Customer.xlsx')

# Convert pandas DataFrame to PySpark DataFrame
customer_df = spark.createDataFrame(df)

# Perform further operations on the Spark DataFrame

/databricks/spark/python/pyspark/sql/pandas/conversion.py:401: UserWarning: createDataFrame attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  Expected bytes, got a 'int' object
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
  warn(msg)


In [0]:
customer_df =customer_df.withColumnRenamed('Customer Id','customer_id')\
    .withColumnRenamed('Customer Name','customer_name')\
    .withColumnRenamed('Segment','segment')\
        .withColumnRenamed('Country','country')\
            .withColumnRenamed('City','city')\
                    .withColumnRenamed('State','state')\
                            .withColumnRenamed('Postal Code','postal_code')\
                                .withColumnRenamed('Region','region')


In [0]:
order_df =spark.read.format('json').option('header',True)\
            .option('inferSchema',True)\
            .option('multiline',True)\
    .load('dbfs:/FileStore/pei/Order.json')

In [0]:
product_schema = 'product_id string, category string,sub_category string, product_name string,state string, price_per_product float'


In [0]:
product_df = spark.read\
        .format('csv')\
            .option('header',True)\
                    .schema(product_schema)\
                    .load('dbfs:/FileStore/pei/Product.csv')

In [0]:
product_df.show(2)

+---------------+----------+------------+--------------------+--------+-----------------+
|     product_id|  category|sub_category|        product_name|   state|price_per_product|
+---------------+----------+------------+--------------------+--------+-----------------+
|FUR-CH-10002961| Furniture|      Chairs|Leather Task Chai...|New York|           81.882|
|TEC-AC-10004659|Technology| Accessories|Imation Secure+ H...|Oklahoma|            72.99|
+---------------+----------+------------+--------------------+--------+-----------------+
only showing top 2 rows



## Source DataFrames 
### customer_df, order_df , product_df

In [0]:
order_df = order_df.withColumnRenamed('Customer ID','customer_id')\
    .withColumnRenamed('Product ID','product_id')\
        .withColumnRenamed('Order Date','order_date')\
        .withColumnRenamed('Discount','discount')\
            .withColumnRenamed('Order ID','order_id')\
                .withColumnRenamed('Profit','profit')\
                    .withColumnRenamed('Quantity','quantity')\
                        .withColumnRenamed('Row ID','row_id')\
                            .withColumnRenamed('Ship Date','ship_date')\
                                .withColumnRenamed('Ship Mode','ship_mode')\





In [0]:
order_df.show(2)

+-----------+--------+----------+--------------+-------+---------------+-------+--------+------+---------+--------------+
|customer_id|discount|order_date|      order_id|  Price|     product_id| profit|quantity|row_id|ship_date|     ship_mode|
+-----------+--------+----------+--------------+-------+---------------+-------+--------+------+---------+--------------+
|   JK-15370|     0.3| 21/8/2016|CA-2016-122581|573.174|FUR-CH-10002961| 63.686|       7|     1|25/8/2016|Standard Class|
|   BD-11320|     0.0| 23/9/2017|CA-2017-117485| 291.96|TEC-AC-10004659|102.186|       4|     2|29/9/2017|Standard Class|
+-----------+--------+----------+--------------+-------+---------------+-------+--------+------+---------+--------------+
only showing top 2 rows



In [0]:
product_new_Df = product_df.na.drop()

In [0]:
print(f'Original dataframe has product_df {product_df.count()} rows and  after removing null we have  product_new_Df with {product_new_Df.count()}  rows ') 


Original dataframe has product_df 1851 rows and  after removing null we have  product_new_Df with 1799  rows 


Profit rounded to 2 decimal places

In [0]:
date_format='yyyy-MM-dd'
order_df= order_df.withColumn('profit', round(col('profit'),2))\
        .withColumn('order_date',trim(col('order_date')))\
            .withColumn('ship_date',trim(col('order_date')))\

        

In [0]:
order_df.show(4)

+-----------+--------+----------+--------------+-------+---------------+------+--------+------+---------+--------------+
|customer_id|discount|order_date|      order_id|  Price|     product_id|profit|quantity|row_id|ship_date|     ship_mode|
+-----------+--------+----------+--------------+-------+---------------+------+--------+------+---------+--------------+
|   JK-15370|     0.3| 21/8/2016|CA-2016-122581|573.174|FUR-CH-10002961| 63.69|       7|     1|21/8/2016|Standard Class|
|   BD-11320|     0.0| 23/9/2017|CA-2017-117485| 291.96|TEC-AC-10004659|102.19|       4|     2|23/9/2017|Standard Class|
|   LB-16795|     0.7| 6/10/2016|US-2016-157490|     17|OFF-BI-10002824|-14.92|       4|     3|6/10/2016|   First Class|
|   KB-16315|     0.2|  2/7/2015|CA-2015-111703| 15.552|OFF-PA-10003349|  5.64|       3|     4| 2/7/2015|Standard Class|
+-----------+--------+----------+--------------+-------+---------------+------+--------+------+---------+--------------+
only showing top 4 rows



Enriching customer name 
Customer name and country


In [0]:
#customer_df_bkp = customer_df
customer_df =customer_df.filter(col('phone') !='#ERROR!').filter(lit(length(col('phone'))>lit(5))).withColumn('phone',regexp_replace('phone','x[0-9]+',''))\
        .withColumn('phone',regexp_replace('phone','[\\.\\-\\(\\)]',''))\
            .withColumn('phone',substring(col('phone'),1,10))
customer_df.show(10)


+-----------+-------------------+--------------------+----------+--------------------+-----------+-------------+----------------+----------+-----------+-------+
|customer_id|      customer_name|               email|     phone|             address|    segment|      country|            city|     state|postal_code| region|
+-----------+-------------------+--------------------+----------+--------------------+-----------+-------------+----------------+----------+-----------+-------+
|   PW-19240|       Pierre Wener|bettysullivan808@...|4215800902|001 Jones Ridges ...|   Consumer|United States|      Louisville|  Colorado|      80027|   West|
|   GH-14410|     Gary567 Hansen|austindyer948@gma...|0015424150|00347 Murphy Unio...|Home Office|United States|         Chicago|  Illinois|      60653|Central|
|   KL-16555|      Kelly Lampkin|clarencehughes280...|7185624866|007 Adams Lane Su...|  Corporate|United States|Colorado Springs|  Colorado|      80906|   West|
|   AH-10075|Ad.       ..am Hart|a

In [0]:
#from pyspark.sql.functions import * 
customer_df = customer_df.withColumn('customer_name',trim('customer_name'))\
.withColumn('customer_name',regexp_replace('customer_name','[@.<>#$%^&*]',''))\
    .withColumn('customer_name',regexp_replace('customer_name','^\\s+|\\s+$',''))\
         .withColumn('customer_name',regexp_replace('customer_name','\\s+',' '))

      

#print(f'old customer_df count is {customer_df.count()} and new customer df count is {new_customer_df.count()}')

Result Set from Question 1 

In [0]:
problem1_1 = order_df
problem1_2 = customer_df.select('customer_name','country')
problem1_3=product_new_Df.select('product_name','sub_category')

In [0]:
order_df.createOrReplaceTempView('orders')

In [0]:
spark.conf.set("spark.sql.parquet.int96RebaseModeInWrite", "CORRECTED")
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")

In [0]:
order_df = order_df.withColumn('order_date', to_date('order_date','dd/MM/yyyy'))\
        .withColumn('ship_date',to_date('ship_date','dd/MM/yyyy'))


In [0]:
order_df.show(4)

+-----------+--------+----------+--------------+-------+---------------+------+--------+------+----------+--------------+----+
|customer_id|discount|order_date|      order_id|  Price|     product_id|profit|quantity|row_id| ship_date|     ship_mode|year|
+-----------+--------+----------+--------------+-------+---------------+------+--------+------+----------+--------------+----+
|   JK-15370|     0.3|2016-08-21|CA-2016-122581|573.174|FUR-CH-10002961| 63.69|       7|     1|2016-08-21|Standard Class|2016|
|   BD-11320|     0.0|2017-09-23|CA-2017-117485| 291.96|TEC-AC-10004659|102.19|       4|     2|2017-09-23|Standard Class|2017|
|   LB-16795|     0.7|2016-10-06|US-2016-157490|     17|OFF-BI-10002824|-14.92|       4|     3|2016-10-06|   First Class|2016|
|   KB-16315|     0.2|2015-07-02|CA-2015-111703| 15.552|OFF-PA-10003349|  5.64|       3|     4|2015-07-02|Standard Class|2015|
+-----------+--------+----------+--------------+-------+---------------+------+--------+------+----------+-----

Create an aggregate table that shows profit by

Year

Product Category

Product Sub Category

Customer


In [0]:
order_df = order_df.withColumn('year', year(col('order_date')))


In [0]:
profit_by_year = order_df.groupBy('year').agg(sum('profit').alias('total'))

In [0]:
customer_df= customer_df.withColumnRenamed('customer_id','customer__id')

In [0]:
merge_df =order_df.join(customer_df ,order_df.customer_id==customer_df.customer__id , how='inner')\
        .join(product_new_Df, on='product_id',how='inner')

In [0]:
merge_df.groupBy('category').agg(round(sum('profit'),2).alias('cat_wise_sum')).show()

+---------------+------------+
|       category|cat_wise_sum|
+---------------+------------+
|Office Supplies|   114968.33|
|      Furniture|      3361.5|
|     Technology|   104157.16|
+---------------+------------+



In [0]:
merge_df.groupBy('sub_category').agg(round(sum('profit'),2).alias('sub_cat_wise_sum')).show()

+------------+----------------+
|sub_category|sub_cat_wise_sum|
+------------+----------------+
|   Envelopes|         5107.39|
|         Art|         5180.91|
|      Chairs|        21508.91|
| Furnishings|          129.33|
|    Supplies|        -1525.05|
|   Fasteners|          626.88|
|     Binders|        33692.77|
|   Bookcases|        -2900.87|
|      Labels|         4785.94|
|       Paper|        26986.03|
| Accessories|        40141.17|
|     Copiers|        47911.96|
|      Phones|        38156.26|
|    Machines|       -22052.23|
|     Storage|         19891.1|
|  Appliances|        20222.36|
|      Tables|       -15375.87|
+------------+----------------+



In [0]:
merge_df.groupBy('customer_id').agg(round(sum('profit'),2).alias('customerprofit')).show()

+-----------+--------------+
|customer_id|customerprofit|
+-----------+--------------+
|   RR-19315|        -73.83|
|   SW-20275|        332.76|
|   PW-19240|       1233.03|
|   IM-15070|        553.89|
|   MG-18145|         82.82|
|   BS-11380|        -21.48|
|   JK-15370|        384.28|
|   NB-18655|        117.64|
|   SS-20515|        210.98|
|   LL-16840|        560.51|
|   AB-10105|       5477.85|
|   AB-10015|        129.68|
|   GK-14620|       1706.18|
|   NK-18490|        121.23|
|   CP-12340|      -1905.17|
|   SC-20680|        360.97|
|   KB-16405|        273.86|
|   MB-17305|        216.37|
|   CS-12400|       -475.39|
|   GM-14695|        188.46|
+-----------+--------------+
only showing top 20 rows



In [0]:
merge_df.createOrReplaceTempView('orders')

In [0]:
spark.sql('select  round(sum(profit),2) as yearly_profit from orders group by  year ').show()

+-------------+
|yearly_profit|
+-------------+
|     54881.87|
|     31832.47|
|     56005.84|
|     79766.81|
+-------------+



In [0]:
spark.sql('select round(sum(profit),2) as Total_profit from orders group by year, category').show()

+------------+
|Total_profit|
+------------+
|    38321.13|
|    21136.43|
|    17953.71|
|    16070.45|
|     3491.93|
|    -6876.42|
|    39879.49|
|     1566.19|
|    30253.51|
|    22638.44|
|      5179.8|
|    32872.33|
+------------+



In [0]:
spark.sql('select round(sum(profit),2)  as customer_profit from orders group by customer_id ').show(5)

+---------------+
|customer_profit|
+---------------+
|         -73.83|
|         332.76|
|        1233.03|
|         553.89|
|          82.82|
+---------------+
only showing top 5 rows



In [0]:
spark.sql('select round(sum(profit),2) as Totalprofit from orders group by customer_id, year').show(5)

+-----------+
|Totalprofit|
+-----------+
|     -23.29|
|      -6.57|
|       9.33|
|     -47.36|
|    -152.72|
+-----------+
only showing top 5 rows

